pca_glove_150features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Glove/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:

df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Glove/glove_withstopwordchangelevels_150_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,0,0.801898,-5.121690,-7.626679,-1.094031,0.580380,3.953755,-2.444268,4.543995,2.839984,...,-0.472631,-0.538083,-0.228704,0.269840,-0.319386,-0.250504,0.171497,-0.317956,-0.081626,0.463571
1,1,-5.189553,-2.617075,-0.547761,-1.667660,5.064081,-2.514712,-0.832970,-1.158165,0.436824,...,0.512073,-0.230609,-0.037974,-0.078831,0.404868,0.191211,-0.498365,0.011369,0.291959,0.255839
2,2,1.528110,1.984331,-2.784060,-6.201825,2.850400,-2.545108,2.340021,-1.455036,-0.338024,...,0.517842,-0.399862,-0.044348,0.992560,0.552784,-0.188813,0.569065,-0.755807,-0.819596,-0.768307
3,3,-4.317900,-0.950158,2.517322,-0.601766,-0.996469,1.206000,2.303441,3.012528,-0.331418,...,0.249997,0.469844,-0.442582,0.195121,-0.086537,0.351137,0.220954,-0.227138,0.231598,0.151744
4,4,-4.712124,4.059922,2.649153,2.424719,1.722507,1.214286,-0.958707,-4.379825,0.715500,...,-0.343802,-0.574985,0.688610,0.250490,-0.715497,-0.102808,0.015435,-0.458648,0.169075,-0.333001


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,0.801898,-5.121690,-7.626679,-1.094031,0.580380,3.953755,-2.444268,4.543995,2.839984,-2.035890,...,-0.472631,-0.538083,-0.228704,0.269840,-0.319386,-0.250504,0.171497,-0.317956,-0.081626,0.463571
1,-5.189553,-2.617075,-0.547761,-1.667660,5.064081,-2.514712,-0.832970,-1.158165,0.436824,1.685180,...,0.512073,-0.230609,-0.037974,-0.078831,0.404868,0.191211,-0.498365,0.011369,0.291959,0.255839
2,1.528110,1.984331,-2.784060,-6.201825,2.850400,-2.545108,2.340021,-1.455036,-0.338024,-3.026875,...,0.517842,-0.399862,-0.044348,0.992560,0.552784,-0.188813,0.569065,-0.755807,-0.819596,-0.768307
3,-4.317900,-0.950158,2.517322,-0.601766,-0.996469,1.206000,2.303441,3.012528,-0.331418,-1.103181,...,0.249997,0.469844,-0.442582,0.195121,-0.086537,0.351137,0.220954,-0.227138,0.231598,0.151744
4,-4.712124,4.059922,2.649153,2.424719,1.722507,1.214286,-0.958707,-4.379825,0.715500,1.234583,...,-0.343802,-0.574985,0.688610,0.250490,-0.715497,-0.102808,0.015435,-0.458648,0.169075,-0.333001


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 150), (524, 150))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3438331291971841


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.433607,0.433607,0.531202,0.416585,0.290076,0.290076,0.36009,0.283178


**Random Forest**

In [23]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [24]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [25]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [26]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9860    0.9723    0.9791       361
           2     0.9835    0.9729    0.9782       553
           3     0.9527    0.9869    0.9695       306

    accuracy                         0.9762      1220
   macro avg     0.9741    0.9774    0.9756      1220
weighted avg     0.9765    0.9762    0.9763      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3103    0.1146    0.1674       157
           2     0.4533    0.8397    0.5888       237
           3     0.2222    0.0462    0.0764       130

    accuracy                         0.4256       524
   macro avg     0.3286    0.3335    0.2775       524
weighted avg     0.3531    0.4256    0.3354       524



In [27]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.433607,0.433607,0.531202,0.416585,0.290076,0.290076,0.360090,0.283178
1,RandomForest,0.976230,0.976230,0.976518,0.976270,0.425573,0.425573,0.353141,0.335420


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6224    0.4931    0.5502       361
           2     0.5892    0.7703    0.6677       553
           3     0.5972    0.4118    0.4874       306

    accuracy                         0.5984      1220
   macro avg     0.6029    0.5584    0.5685      1220
weighted avg     0.6010    0.5984    0.5877      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2712    0.2038    0.2327       157
           2     0.4633    0.6118    0.5273       237
           3     0.3118    0.2231    0.2601       130

    accuracy                         0.3931       524
   macro avg     0.3488    0.3462    0.3400       524
weighted avg     0.3681    0.3931    0.3727       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.520492,0.520492,0.513125,0.504761,0.395038,0.395038,0.361363,0.364309
1,RandomForest,0.976230,0.976230,0.976314,0.976249,0.414122,0.414122,0.349559,0.338013
2,Adaboost,0.598361,0.598361,0.601018,0.587730,0.393130,0.393130,0.368142,0.372736


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9881    0.9197    0.9527       361
           2     0.9118    0.9910    0.9497       553
           3     0.9788    0.9052    0.9406       306

    accuracy                         0.9484      1220
   macro avg     0.9596    0.9386    0.9477      1220
weighted avg     0.9512    0.9484    0.9483      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2358    0.1847    0.2071       157
           2     0.4593    0.6667    0.5439       237
           3     0.2807    0.1231    0.1711       130

    accuracy                         0.3874       524
   macro avg     0.3253    0.3248    0.3074       524
weighted avg     0.3480    0.3874    0.3505       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.520492,0.520492,0.513125,0.504761,0.395038,0.395038,0.361363,0.364309
1,RandomForest,0.976230,0.976230,0.976314,0.976249,0.414122,0.414122,0.349559,0.338013
2,Adaboost,0.598361,0.598361,0.601018,0.587730,0.393130,0.393130,0.368142,0.372736
3,GBM,0.948361,0.948361,0.951187,0.948304,0.387405,0.387405,0.348019,0.350514


**SVM Model**

In [28]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [29]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [30]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [31]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4990    0.6593    0.5680       361
           2     0.6935    0.4828    0.5693       553
           3     0.5112    0.5980    0.5512       306

    accuracy                         0.5639      1220
   macro avg     0.5679    0.5800    0.5628      1220
weighted avg     0.5902    0.5639    0.5644      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2692    0.3121    0.2891       157
           2     0.4725    0.3629    0.4105       237
           3     0.2437    0.3000    0.2690       130

    accuracy                         0.3321       524
   macro avg     0.3285    0.3250    0.3229       524
weighted avg     0.3549    0.3321    0.3390       524



In [32]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.433607,0.433607,0.531202,0.416585,0.290076,0.290076,0.360090,0.283178
1,RandomForest,0.976230,0.976230,0.976518,0.976270,0.425573,0.425573,0.353141,0.335420
2,SVM,0.563934,0.563934,0.590205,0.564381,0.332061,0.332061,0.354858,0.339009


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5342    0.5623    0.5479       361
           2     0.5870    0.7197    0.6466       553
           3     0.4938    0.2614    0.3419       306

    accuracy                         0.5582      1220
   macro avg     0.5384    0.5145    0.5121      1220
weighted avg     0.5480    0.5582    0.5410      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3013    0.2994    0.3003       157
           2     0.4300    0.5443    0.4804       237
           3     0.2647    0.1385    0.1818       130

    accuracy                         0.3702       524
   macro avg     0.3320    0.3274    0.3209       524
weighted avg     0.3504    0.3702    0.3524       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.520492,0.520492,0.513125,0.504761,0.395038,0.395038,0.361363,0.364309
1,RandomForest,0.976230,0.976230,0.976314,0.976249,0.414122,0.414122,0.349559,0.338013
2,Adaboost,0.598361,0.598361,0.601018,0.587730,0.393130,0.393130,0.368142,0.372736
3,GBM,0.948361,0.948361,0.951187,0.948304,0.387405,0.387405,0.348019,0.350514
4,SVM,0.581967,0.581967,0.584153,0.564323,0.374046,0.374046,0.339882,0.348041
5,KNN,0.558197,0.558197,0.548019,0.540980,0.370229,0.370229,0.350426,0.352390


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4914    0.5540    0.5208       361
           2     0.6045    0.5859    0.5950       553
           3     0.4801    0.4346    0.4563       306

    accuracy                         0.5385      1220
   macro avg     0.5253    0.5249    0.5240      1220
weighted avg     0.5398    0.5385    0.5383      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2722    0.2739    0.2730       157
           2     0.4580    0.5063    0.4810       237
           3     0.2596    0.2077    0.2308       130

    accuracy                         0.3626       524
   macro avg     0.3299    0.3293    0.3282       524
weighted avg     0.3531    0.3626    0.3566       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.520492,0.520492,0.513125,0.504761,0.395038,0.395038,0.361363,0.364309
1,RandomForest,0.976230,0.976230,0.976314,0.976249,0.414122,0.414122,0.349559,0.338013
2,Adaboost,0.598361,0.598361,0.601018,0.587730,0.393130,0.393130,0.368142,0.372736
3,GBM,0.948361,0.948361,0.951187,0.948304,0.387405,0.387405,0.348019,0.350514
4,SVM,0.581967,0.581967,0.584153,0.564323,0.374046,0.374046,0.339882,0.348041
5,KNN,0.558197,0.558197,0.548019,0.540980,0.370229,0.370229,0.350426,0.352390
6,GNB,0.538525,0.538525,0.539833,0.538274,0.362595,0.362595,0.353106,0.356587
